# Web Scraping with Beautiful Soup

## Introduction

Web scraping is a technique to automatically access and extract large amounts of information from a website, which can save a huge amount of time and effort. In this kernel, I will introduce you to web scraping using the Python programming language and the Beautiful Soup library.

## Beautiful Soup

Beautiful Soup is a Python library for pulling data out of HTML and XML files. It creates a parse tree from page source code that can be used to extract data easily. It is a powerful library and very easy to use.

In [1]:
from datetime import datetime
print(f"Hello, scrapers {datetime.now()}")
import sys
print(f"Python version: {sys.version}")

#  Requests for handling HTTP get and other requests
try:
    import requests
    # print version
    print(f"Requests version: {requests.__version__}")
except ImportError:
    print("requests is not installed")
    print("Install it with pip install requests")

# requests is external library
# if you used Python locally you would need to install it with pip install requests
import time # import for playing nice and not getting blocked

import pandas as pd # for data analysis - more on pandas will be in other lecture
from datetime import datetime # standard library for date and time
# from BeautifulSoup4 import BeatifulSoup if installed through pip install BeautifulSoup4
#
try:
    from bs4 import BeautifulSoup
    from bs4 import __version__ as bs4_version
    # print version
    print(f"BeautifulSoup version: {bs4_version}")
    # print(f"BeautifulSoup version: {BeautifulSoup.__version__}")
except ImportError:
    print("BeautifulSoup is not installed")
    print("Install it with pip install BeautifulSoup4")
# from bs4 import BeautifulSoup # an external library for parsing HTML (and XML) content
# bs4 is external library as well

# all of the above should be available on Google Colab as well as your local machine

Hello, scrapers 2025-12-04 20:54:32.227490
Python version: 3.13.9 (tags/v3.13.9:8183fa5, Oct 14 2025, 14:09:13) [MSC v.1944 64 bit (AMD64)]
Requests version: 2.32.5
BeautifulSoup version: 4.14.3


In [ ]:
# !pip install beautifulsoup4
# if bs4 not found install it with

# https://www.crummy.com/software/BeautifulSoup/

In [2]:
# first we get the url we want to scrape
# centrs = "https://www.ss.com/lv/real-estate/flats/riga/centre/sell/"
centrs = "https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/"
# centrs = "https://www.ss.com/en/real-estate/flats/riga/centre/sell/"
# centrs
# agenskalns = "https://www.ss.com/en/real-estate/flats/riga/agenskalns/sell/"

# url = agenskalns # just an alias
url = centrs # just an alias
print(f"URL: {url}")


URL: https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/


In [3]:
# here we make a something called HTTP GET request to the url in centrs
# so very similar to what the browser does
print(f"Requesting URL: {url}")
response = requests.get(url) # here we make a request to the url in centrs, and store the response in req
print(f"Response code: {response.status_code}")
# so where we could use our status_code to check if our request went through sucessfully
# so 200 is the ALL OK code in HTTP, many 4xx and 5xxx failure codes

Requesting URL: https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/
Response code: 200


In [ ]:
# list of HTTP response codes
# https://en.wikipedia.org/wiki/List_of_HTTP_status_codes

## Seeing text of the response

The text of the response will be HTML, which is a markup language that is used to create web pages. Let's see the text of the response.

In [4]:
# so now we can start working with the response without making additional requests
response.text[:300] # first 300 text charactersb

'<!DOCTYPE html>\r\n<HTML lang="en"><HEAD>\r\n<title>SS.COM Flats - Riga - Centre, Prices, Hand over - Advertisements</title>\r\n<meta http-equiv="Content-Type" CONTENT="text/html; charset=UTF-8">\r\n<meta name="viewport" content="user-scalable=1, width=device-width, initial-scale=1.0"/>\r\n<meta name="keyword'

In [5]:
# we could attempt to parse the text with the standard library, but it is not very time consuming
# basically a "fool's errand" - because we have better tools for the job - BeautifulSoup
"Agensk" in response.text

True

In [6]:
response.text.index("Eliz")

14127

In [7]:
my_index = response.text.index("Eliz")
# So Eliz needle is at
my_index

14127

In [8]:
# i could us slicing to see the concordance
window_size = 150
response.text[my_index-window_size:my_index+window_size] # getting some text around what I was looking forb

'">Duntes</option><option value="4161">Dzirnavu</option><option value="4175">Eksporta</option><option value="4178">Elijas</option><option value="4179">Elizabetes</option><option value="4181">Elku</option><option value="4225">Ganibu d.</option><option value="4230">Ganu</option><option value="4281">Ger'

In [ ]:
# we could actually get the whole table with pandas library
# this approach requires that data on the web page exists within <table> ..... </table> tags

In [ ]:
# we could manually use text searching tools to work on the whole text of html but it would not convenient
# many other people have already written tools to parse/structure this text

## Using Beautiful Soup for parsing HTML


In [9]:
# parser means structuring your text into some data structure
# we pass the text of the response to the parser
soup = BeautifulSoup(response.text, 'lxml') # lxml is improved parser, a little bit more lenient towards badly structured HTML
# you could also use default parser with command
# soup = BeautifulSoup(req.text) - but it is slower and less lenient
# soup = BeautifulSoup(req.text)
# note this process does not require internet connection, it is just parsing the text
# the internet connection was only needed to get the response first
soup.title

<title>SS.COM Flats - Riga - Centre, Prices, Hand over - Advertisements</title>

In [15]:
type(soup)  # so we have soup object which contains all the information about the page in a structured manner
# of course you could use another name for the object, but soup is a convention

bs4.BeautifulSoup

## Finding element which contains last page number

We know it is an anchor element with some atrributes. Let's find it.

In [16]:
# we could use find_all method to find all anchor elements
# anchor elements are links
anchors = soup.find_all("a") # so we are looking for all <a> elements in the page
print(f"Found {len(anchors)} anchor elements")

Found 99 anchor elements


In [17]:
# well 96 is a big number we could see if we could filter them out
# in this case we know our anchor has specific attribute rel="prev" so we could filter them out
# in this case I will use find method which finds first matching element
last_page_anchor = soup.find("a", {"rel": "prev"}) # so we are looking for all <a> elements with attribute rel="prev"
# note how we passed in dictionary to specify the attribute or attributes we are looking for
last_page_anchor

<a class="navi" href="/en/real-estate/flats/riga/centre/hand_over/page25.html" name="nav_id" rel="prev"><img border="0" height="5" src="https://i.ss.com/img/s_left.png" style="padding-bottom:2px;" width="9"/> Previous</a>

In [18]:
# let's get href attribute of the anchor
last_page_href = last_page_anchor.get("href") # so this get is like getting a value from a dictionary
print(f"Last page href: {last_page_href}")

Last page href: /en/real-estate/flats/riga/centre/hand_over/page25.html


In [19]:
# now let's get the number of the last page we could use regular expression but here we will use string manipulation
# we will split the string by / and get the last element
last_part = last_page_href.split("page")[-1] # so we are splitting the string by / and getting the second to last element
print(f"Last part: {last_part}")

Last part: 25.html


In [20]:
# now we can split by dot and get the number_str
number_str = last_part.split(".")[0] # so we are splitting the string by . and getting the first element
print(f"Number string: {number_str}")

Number string: 25


In [21]:
# now we just need to convert to integer
last_page_number = int(number_str) # so we are converting string to integer
print(f"Last page number: {last_page_number}")

Last page number: 25


In [22]:
url

'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/'

In [24]:
# now we need to create a function that takes a url and returns a list of all urls including he original one
# so we will create a function that takes a url and returns a list of urls
def get_urls(url):
    # get response
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to get response from {url}")
        return [] # means we failed to have a good response
    print(f"Successfully got response from {url}")
    # create soup
    soup = BeautifulSoup(response.text, 'lxml')
    # find anchor with rel=prev
    last_page_anchor = soup.find("a", {"rel": "prev"})
    # check if it is not None
    if last_page_anchor is None:
        return [url] # means there are no more pages, means we just need one page
    # get href
    last_page_href = last_page_anchor.get("href")
    # get last part
    last_part = last_page_href.split("page")[-1]
    # get number string
    number_str = last_part.split(".")[0]
    # get last page number
    last_page_number = int(number_str)
    # now we can build the full list of urls
    urls = [url] # we start with the original url
    for i in range(2, last_page_number+1):
        urls.append(f"{url}page{i}.html") # note our url includes / at the end

    return urls

# let's test our function
# we will test it on the first page of agenskalns
urls = get_urls(url)
print(f"URLs: {urls}")

Successfully got response from https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/
URLs: ['https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/', 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page2.html', 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page3.html', 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page4.html', 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page5.html', 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page6.html', 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page7.html', 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page8.html', 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page9.html', 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page10.html', 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page11.html', 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page12.h

In [ ]:
# we could now use pandas to loop through all the urls and get the tables
# however we want to get the actual links to the ads so we will continue using BeautifulSoup

In [25]:
# https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all
tables = soup.find_all("table") # finds ALL elements matching our filter
len(tables) # it will be a list of some length, possibly 0 if we did not find any elements

6

In [ ]:
# Now we want to find what our column names should be no matter the type of ad
# tr stands for table row
# mdn tr documentation
# https://developer.mozilla.org/en-US/docs/Web/HTML/Element/tr

In [26]:
# find finds just one(first) match
headline = soup.find("tr", {"id":"head_line"}) # this is a shorter way of finding by tr AND this element having particular id
headline

<tr id="head_line">
<td class="msg_column" colspan="3" width="70%">
<span style="float:left;"> Advertisements
</span>
<span align="right" class="msg_column" style="float:right;text-align:right;padding-right:3px;">
<noindex>
<a class="a19" href="/en/real-estate/flats/riga/centre/hand_over/fDgSeF4S.html" rel="nofollow">date</a></noindex></span>
</td>
<td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/en/real-estate/flats/riga/centre/hand_over/fDgSeF4SFDwT.html" rel="nofollow" title="">Street</a></noindex></td><td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/en/real-estate/flats/riga/centre/hand_over/fDgSeF4SelM=.html" rel="nofollow" title="">R.</a></noindex></td><td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/en/real-estate/flats/riga/centre/hand_over/fDgSeF4QelM=.html" rel="nofollow" title="">m²</a></noindex></td><td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/en/real-estate/flats/riga/centre/hand_over/fDgSeF4XelM=.

In [27]:
type(headline)

bs4.element.Tag

In [28]:
# next we can search through the headling table row element and find individual table data elements
headtds = headline.find_all("td") #note I am looking through not ALL soup object but just the headline element
# https://developer.mozilla.org/en-US/docs/Web/HTML/Element/td
headtds  # a list of td elements

[<td class="msg_column" colspan="3" width="70%">
 <span style="float:left;"> Advertisements
 </span>
 <span align="right" class="msg_column" style="float:right;text-align:right;padding-right:3px;">
 <noindex>
 <a class="a19" href="/en/real-estate/flats/riga/centre/hand_over/fDgSeF4S.html" rel="nofollow">date</a></noindex></span>
 </td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/en/real-estate/flats/riga/centre/hand_over/fDgSeF4SFDwT.html" rel="nofollow" title="">Street</a></noindex></td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/en/real-estate/flats/riga/centre/hand_over/fDgSeF4SelM=.html" rel="nofollow" title="">R.</a></noindex></td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/en/real-estate/flats/riga/centre/hand_over/fDgSeF4QelM=.html" rel="nofollow" title="">m²</a></noindex></td>,
 <td class="msg_column_td" nowrap=""><noindex><a class="a18" href="/en/real-estate/flats/riga/centre/hand_over/fDgSeF4XelM=.ht

In [29]:
len(headtds) # so we have some of lements (could be 8, 10 depends on the type of ad)

8

In [30]:
headline.text # the text is extracted from children, grandchildren, greatgrandchildren elements and so on

'\n\n\xa0Advertisements\n\n\n\ndate\n\nStreetR.m²FloorSeriesPrice, m2Price'

In [31]:
headtds[3].text  # so text of 4th child of headline (which is tr with id headline) element

'm²'

In [32]:
# so we do not want announcements and data so we will skip the first TD with headtds[1:]
headcolumns = [el.text for el in headtds[1:]] #.text gets us content even from children and grandchildren
headcolumns

['Street', 'R.', 'm²', 'Floor', 'Series', 'Price, m2', 'Price']

In [34]:
# lets combine the above cells into a function which will always get us columns
def getColList(soup, default_columns = ("description","url")):
    column_list = list(default_columns) # we decided to that we need these two column names no matter the html
    headline = soup.find("tr", {"id":"head_line"})
    headtds = headline.find_all("td")
    headcolumns = [el.text for el in headtds[1:]] # this will get all column names starting with 2nd in HTML
    column_list += headcolumns # we add the head columns to our initial column list
    return column_list

In [35]:
# now that I have a function I can use it on any soup object
column_names = getColList(soup)
column_names # so this should work no matter what type of SS.com page the soup is made of

['description',
 'url',
 'Street',
 'R.',
 'm²',
 'Floor',
 'Series',
 'Price, m2',
 'Price']

In [36]:
trows = soup.find_all('tr')  # so i want all table rows on the page
print(f"Got {len(trows)} table rows") # i get more than 30 rows, so I need to filter them


Got 39 table rows


In [37]:
print(trows[5]) # our first ad so from trows[5] to trows[34 would work as well]

<tr id="tr_57005579"><td class="msga2 pp0"><input id="c57005579" name="mid[]" type="checkbox" value="57005579_1106_0"/></td><td class="msga2"><a href="/msg/en/real-estate/flats/riga/centre/adbhd.html" id="im57005579"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/8/1410/352301/70460109.th2.jpg"/></a></td><td class="msg2"><div class="d1"><a class="am" data="eWYlODAlQUElQTN1JUUwdWclODUlQTYlQTN6JTk2JTdCZyU4OSVBQSVBMCU3QiU5NHhhJTgwJUEy|D1PrmCd" href="/msg/en/real-estate/flats/riga/centre/adbhd.html" id="dm_57005579">2 storey flat in a renovated house. Comes fully furnished and eq</a></div></td><td c="1" class="msga2-o pp6" nowrap="">Birznieka-Upisha 1..</td><td c="1" class="msga2-o pp6" nowrap="">4</td><td c="1" class="msga2-o pp6" nowrap="">108</td><td c="1" class="msga2-o pp6" nowrap="">5/6</td><td c="1" class="msga2-o pp6" nowrap="">Recon.</td><td c="1" class="msga2-o pp6" nowrap="">11.11 €</td><td c="1" class="msga2-o pp6" nowrap="">1,200  €/mon.</td></tr>


In [38]:
# less flexible solution would be to hard code this
all_ads = trows[5:35] # so starting from 6th element to 35th element
all_ads[-1] # last one - 35

<tr id="tr_57130958"><td class="msga2 pp0"><input id="c57130958" name="mid[]" type="checkbox" value="57130958_1106_0"/></td><td class="msga2"><a href="/msg/en/real-estate/flats/riga/centre/abfkc.html" id="im57130958"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/8/1441/360025/72004958.th2.jpg"/></a></td><td class="msg2"><div class="d1"><a class="am" data="JTk0JUE1Zm0lOUMlOUMlOUIlRjFibCU5NiU5QiU5QyVBRGlrJTlDJUEwJTk5JUFFZ2UlOTglOTclOTMlQTU=|cu15cg" href="/msg/en/real-estate/flats/riga/centre/abfkc.html" id="dm_57130958">Mansarda dzīvoklis īrei. 
Fasādes māja, renovēta māja, ieeja no</a></div></td><td c="1" class="msga2-o pp6" nowrap="">Laboratorijas 11</td><td c="1" class="msga2-o pp6" nowrap="">2</td><td c="1" class="msga2-o pp6" nowrap="">32</td><td c="1" class="msga2-o pp6" nowrap="">5/5</td><td c="1" class="msga2-o pp6" nowrap="">Pre-war house</td><td c="1" class="msga2-o pp6" nowrap="">8.13 €</td><td c="1" class="msga2-o pp6" nowrap="">260  €/mon.</td></tr>

In [ ]:
# if i had no other choices I could find starting and end index by hand and hard code it
# but that is liable to change between pages

In [39]:
# hardest part in this
# how to filter only specific rows
# we need to find something that is unique to our apartment rows but not to the extra rows
# remember we id is not guaranteed, so "" gives us default value when there is no id
# because it is possible there is no id atrribute at all
# sometimes there were unusable rows with tr_bnr attribute
# in this recipe you would change the startswith to whatever you need in your case
# and not row.get('id',"").startswith("tr_bnr") is not needed anymore but left for legacy reasons
apt_rows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
# note the use of get method to get the value of the id attribute
# table rows are not guaranteed to have id attribute
# so we use the trick of getting the value of the attribute or empty string if it does not exist
# some pages have this fake tr with id tr_bnr element which are not ads
# you could have used a regular loop based solution but this one liner is more elegant
len(apt_rows)

30

In [40]:
apt_rows[-1]

<tr id="tr_57130958"><td class="msga2 pp0"><input id="c57130958" name="mid[]" type="checkbox" value="57130958_1106_0"/></td><td class="msga2"><a href="/msg/en/real-estate/flats/riga/centre/abfkc.html" id="im57130958"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/8/1441/360025/72004958.th2.jpg"/></a></td><td class="msg2"><div class="d1"><a class="am" data="JTk0JUE1Zm0lOUMlOUMlOUIlRjFibCU5NiU5QiU5QyVBRGlrJTlDJUEwJTk5JUFFZ2UlOTglOTclOTMlQTU=|cu15cg" href="/msg/en/real-estate/flats/riga/centre/abfkc.html" id="dm_57130958">Mansarda dzīvoklis īrei. 
Fasādes māja, renovēta māja, ieeja no</a></div></td><td c="1" class="msga2-o pp6" nowrap="">Laboratorijas 11</td><td c="1" class="msga2-o pp6" nowrap="">2</td><td c="1" class="msga2-o pp6" nowrap="">32</td><td c="1" class="msga2-o pp6" nowrap="">5/5</td><td c="1" class="msga2-o pp6" nowrap="">Pre-war house</td><td c="1" class="msga2-o pp6" nowrap="">8.13 €</td><td c="1" class="msga2-o pp6" nowrap="">260  €/mon.</td></tr>

In [41]:
apt_rows[0]

<tr id="tr_57005579"><td class="msga2 pp0"><input id="c57005579" name="mid[]" type="checkbox" value="57005579_1106_0"/></td><td class="msga2"><a href="/msg/en/real-estate/flats/riga/centre/adbhd.html" id="im57005579"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/8/1410/352301/70460109.th2.jpg"/></a></td><td class="msg2"><div class="d1"><a class="am" data="eWYlODAlQUElQTN1JUUwdWclODUlQTYlQTN6JTk2JTdCZyU4OSVBQSVBMCU3QiU5NHhhJTgwJUEy|D1PrmCd" href="/msg/en/real-estate/flats/riga/centre/adbhd.html" id="dm_57005579">2 storey flat in a renovated house. Comes fully furnished and eq</a></div></td><td c="1" class="msga2-o pp6" nowrap="">Birznieka-Upisha 1..</td><td c="1" class="msga2-o pp6" nowrap="">4</td><td c="1" class="msga2-o pp6" nowrap="">108</td><td c="1" class="msga2-o pp6" nowrap="">5/6</td><td c="1" class="msga2-o pp6" nowrap="">Recon.</td><td c="1" class="msga2-o pp6" nowrap="">11.11 €</td><td c="1" class="msga2-o pp6" nowrap="">1,200  €/mon.</td></tr>

In [42]:
# lets make a function from the above doodle and make it work on most pages on SS
def getRowList(soup):
    trows = soup.find_all('tr')
    aprows = [row for row in trows if row.get('id',"").startswith("tr_") and not row.get('id',"").startswith("tr_bnr") ]
    return aprows

# check if it works
rows = getRowList(soup)
print(f"Got {len(rows)} rows")

Got 30 rows


In [43]:
# we are going to check the first ad

row_tds = apt_rows[0].find_all('td') # get all table data elements from first ad/row
len(row_tds)

10

In [44]:
row_tds

[<td class="msga2 pp0"><input id="c57005579" name="mid[]" type="checkbox" value="57005579_1106_0"/></td>,
 <td class="msga2"><a href="/msg/en/real-estate/flats/riga/centre/adbhd.html" id="im57005579"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/8/1410/352301/70460109.th2.jpg"/></a></td>,
 <td class="msg2"><div class="d1"><a class="am" data="eWYlODAlQUElQTN1JUUwdWclODUlQTYlQTN6JTk2JTdCZyU4OSVBQSVBMCU3QiU5NHhhJTgwJUEy|D1PrmCd" href="/msg/en/real-estate/flats/riga/centre/adbhd.html" id="dm_57005579">2 storey flat in a renovated house. Comes fully furnished and eq</a></div></td>,
 <td c="1" class="msga2-o pp6" nowrap="">Birznieka-Upisha 1..</td>,
 <td c="1" class="msga2-o pp6" nowrap="">4</td>,
 <td c="1" class="msga2-o pp6" nowrap="">108</td>,
 <td c="1" class="msga2-o pp6" nowrap="">5/6</td>,
 <td c="1" class="msga2-o pp6" nowrap="">Recon.</td>,
 <td c="1" class="msga2-o pp6" nowrap="">11.11 €</td>,
 <td c="1" class="msga2-o pp6" nowrap="">1,200  €/mon.</td>]

In [45]:
# this is the checkbox we want nothing from that
row_tds[0].text

''

In [46]:
# second is the image table data cell
row_tds[1].text

''

In [47]:
row_tds[1].attrs

{'class': ['msga2']}

In [48]:
img = row_tds[1].find("img")
img

<img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/8/1410/352301/70460109.th2.jpg"/>

In [49]:
img.get("src")

'https://i.ss.com/gallery/8/1410/352301/70460109.th2.jpg'

In [50]:
row_tds[2].text

'2 storey flat in a renovated house. Comes fully furnished and eq'

In [51]:
# last one should be price
row_tds[-1].text # notice this might need some cleaning to convert to number

'1,200  €/mon.'

In [52]:
# so second table data cell (which has alos has image) has anchor
a = row_tds[1].find('a') # a tag is called anchor tag
a

<a href="/msg/en/real-estate/flats/riga/centre/adbhd.html" id="im57005579"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/8/1410/352301/70460109.th2.jpg"/></a>

In [53]:
a.attrs  # so i can read attributes as a dictionary

{'href': '/msg/en/real-estate/flats/riga/centre/adbhd.html',
 'id': 'im57005579'}

In [54]:
a.get('href'), a['href'] # two ways of accesing this attribute
# so all we need is to add http://www.ss.com in front

('/msg/en/real-estate/flats/riga/centre/adbhd.html',
 '/msg/en/real-estate/flats/riga/centre/adbhd.html')

In [55]:
column_names

['description',
 'url',
 'Street',
 'R.',
 'm²',
 'Floor',
 'Series',
 'Price, m2',
 'Price']

In [56]:
# combining all of the above work into a function
def getRow(row,colist=column_names):
    """
    row is a soup object representing a row of the table - tr element
    inside row should be some table data elements which we will extract
    along with the url and description
    colist is a list of column names
    """
    row_tds = row.find_all('td')  # find all table data elements
    rowDict = {} # we will store our data in a dictionary, so we can easily convert it to dataframe
    if len(row_tds) <3: # a little sanity check, if we have less than 3 table data elements we will skip this row
        print("Hmm bad row")
        return rowDict
    # first we save description
    rowDict[colist[0]] = row_tds[2].text # so the big assumption is that we always get description in 3rd column
    # then link to the ad from the image in the second column
    rowDict[colist[1]] = "https://ss.com" + row_tds[1].find('a').get('href')
    # then we take the rest of the table data elements and use them as values for our dictionary with keys being the column names
    for td,key in zip(row_tds[3:],colist[2:]): # we zip two lists together and iterate over them
        rowDict[key] = td.text
    return rowDict

In [57]:
apt_rows[0] # first row of ads

<tr id="tr_57005579"><td class="msga2 pp0"><input id="c57005579" name="mid[]" type="checkbox" value="57005579_1106_0"/></td><td class="msga2"><a href="/msg/en/real-estate/flats/riga/centre/adbhd.html" id="im57005579"><img alt="" class="isfoto foto_list" src="https://i.ss.com/gallery/8/1410/352301/70460109.th2.jpg"/></a></td><td class="msg2"><div class="d1"><a class="am" data="eWYlODAlQUElQTN1JUUwdWclODUlQTYlQTN6JTk2JTdCZyU4OSVBQSVBMCU3QiU5NHhhJTgwJUEy|D1PrmCd" href="/msg/en/real-estate/flats/riga/centre/adbhd.html" id="dm_57005579">2 storey flat in a renovated house. Comes fully furnished and eq</a></div></td><td c="1" class="msga2-o pp6" nowrap="">Birznieka-Upisha 1..</td><td c="1" class="msga2-o pp6" nowrap="">4</td><td c="1" class="msga2-o pp6" nowrap="">108</td><td c="1" class="msga2-o pp6" nowrap="">5/6</td><td c="1" class="msga2-o pp6" nowrap="">Recon.</td><td c="1" class="msga2-o pp6" nowrap="">11.11 €</td><td c="1" class="msga2-o pp6" nowrap="">1,200  €/mon.</td></tr>

In [58]:
getRow(apt_rows[0])

{'description': '2 storey flat in a renovated house. Comes fully furnished and eq',
 'url': 'https://ss.com/msg/en/real-estate/flats/riga/centre/adbhd.html',
 'Street': 'Birznieka-Upisha 1..',
 'R.': '4',
 'm²': '108',
 'Floor': '5/6',
 'Series': 'Recon.',
 'Price, m2': '11.11 €',
 'Price': '1,200  €/mon.'}

In [59]:
# so if we can get a row we can get all rows
def getRows(rowlist,colist=column_names):
    """
    rowlist is a list of soup objects representing rows of the table
    colist is a list of column names
    """
    # return [getRow(row, colist=colist) for row in rowlist] # list comprehension
    # we could also do regular loop
    rowDicts = [] # we will store our data in a list of dictionaries
    for row in rowlist:
        rowDicts.append(getRow(row, colist=colist))
    return rowDicts

# so we supply the row elements and out comes the list of dictionaries with the information about the ads


In [60]:
row_ads = getRows(apt_rows)
row_ads[-3:] # check last three ads

[{'description': 'Namsaimnieks izīrē omulīgu, labiekārtotu, siltu un saulainu 2 is',
  'url': 'https://ss.com/msg/en/real-estate/flats/riga/centre/ehipm.html',
  'Street': 'Stabu 21',
  'R.': '2',
  'm²': '70',
  'Floor': '4/6',
  'Series': 'Pre-war house',
  'Price, m2': '11.71 €',
  'Price': '820  €/mon.'},
 {'description': 'Izīrē burvīgu divistabu dzīvokli (two room apartment) Rīgas klus',
  'url': 'https://ss.com/msg/en/real-estate/flats/riga/centre/dhxdl.html',
  'Street': 'Baznicas 8',
  'R.': '2',
  'm²': '67',
  'Floor': '1/5',
  'Series': 'Pre-war house',
  'Price, m2': '9.70 €',
  'Price': '650  €/mon.'},
 {'description': 'Mansarda dzīvoklis īrei. \nFasādes māja, renovēta māja, ieeja no',
  'url': 'https://ss.com/msg/en/real-estate/flats/riga/centre/abfkc.html',
  'Street': 'Laboratorijas 11',
  'R.': '2',
  'm²': '32',
  'Floor': '5/5',
  'Series': 'Pre-war house',
  'Price, m2': '8.13 €',
  'Price': '260  €/mon.'}]

In [ ]:
type(row_ads) # so a list of dictionaries well suited for conversion to Pandas

list

In [61]:
row_ads[:2]  # first two ads

[{'description': '2 storey flat in a renovated house. Comes fully furnished and eq',
  'url': 'https://ss.com/msg/en/real-estate/flats/riga/centre/adbhd.html',
  'Street': 'Birznieka-Upisha 1..',
  'R.': '4',
  'm²': '108',
  'Floor': '5/6',
  'Series': 'Recon.',
  'Price, m2': '11.11 €',
  'Price': '1,200  €/mon.'},
 {'description': 'Fasādes māja, bezmaksas stāvvieta pagalmā, ieeja no pagalma, log',
  'url': 'https://ss.com/msg/en/real-estate/flats/riga/centre/dhpnc.html',
  'Street': 'Pernavas 11',
  'R.': '2',
  'm²': '65',
  'Floor': '6/7',
  'Series': 'Pre-war house',
  'Price, m2': '5.38 €',
  'Price': '350  €/mon.'}]

In [62]:
# we can pass our list of dictionaries to pandas and create a dataframe
dtemp = pd.DataFrame(row_ads, columns=column_names)
dtemp.shape

(30, 9)

In [63]:
dtemp.head()

,description,url,Street,R.,m²,Floor,Series,"Price, m2",Price
0,2 storey flat in a renovated house. Comes full...,https://ss.com/msg/en/real-estate/flats/riga/c...,Birznieka-Upisha 1..,4,108,5/6,Recon.,11.11 €,"1,200 €/mon."
1,"Fasādes māja, bezmaksas stāvvieta pagalmā, iee...",https://ss.com/msg/en/real-estate/flats/riga/c...,Pernavas 11,2,65,6/7,Pre-war house,5.38 €,350 €/mon.
2,"Fasādes māja, slēgts pagalms, ieeja no ielas u...",https://ss.com/msg/en/real-estate/flats/riga/c...,Baznicas 45,2,50,1/6,Pre-war house,5.20 €,260 €/mon.
3,Īrei tiek piedāvāts 3 istabu dzīvoklis Rīgas c...,https://ss.com/msg/en/real-estate/flats/riga/c...,Marijas 16,3,90,2/6,Pre-war house,7.78 €,700 €/mon.
4,Īrei tiek piedāvāts moderns dzīvoklis eklektis...,https://ss.com/msg/en/real-estate/flats/riga/c...,Stabu 30,2,53,3/4,Pre-war house,9.81 €,520 €/mon.


In [64]:
# now you can save it to a file in a format you like, such as csv or excel
# dtemp.to_excel("centrs_30.xlsx")
# aganskalns_30.xlsx
# dtemp.to_excel("aganskalns_30.xlsx")
dtemp.to_excel("centers_04122025.xlsx")

In [67]:
# we have the list of urls already!
# let's call our function on all of them
# and combine the results into one dataframe
# let's make a function for this and also provid a small delay between requests
def get_dataframe_from_url(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to get response from {url}")
        return None
    print(f"Successfully got response from {url}")
    soup = BeautifulSoup(response.text, 'lxml')
    rows = getRowList(soup)
    row_ads = getRows(rows)
    return pd.DataFrame(row_ads, columns=column_names)

# now for all
def get_big_dataframe(urls):
    url_list = []
    for url in urls:
        print(f"Getting data from {url}")
        df = get_dataframe_from_url(url)
        if df is not None:
            url_list.append(df)
        time.sleep(0.3) # sleep for a second or less here 300 milliseconds
    big_df = pd.concat(url_list, ignore_index=True)
    return big_df

In [68]:
urls # we made this list earlier

['https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page2.html',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page3.html',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page4.html',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page5.html',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page6.html',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page7.html',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page8.html',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page9.html',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page10.html',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page11.html',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page12.html',
 'https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page13.html',
 'htt

In [69]:
big_df = get_big_dataframe(urls)

Getting data from https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/
Successfully got response from https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/
Getting data from https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page2.html
Successfully got response from https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page2.html
Getting data from https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page3.html
Successfully got response from https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page3.html
Getting data from https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page4.html
Successfully got response from https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page4.html
Getting data from https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page5.html
Successfully got response from https://www.ss.com/en/real-estate/flats/riga/centre/hand_over/page5.html
Getting data from https://www.ss.com/en/real-

In [70]:
# let's see shape and head
print(f"Shape: {big_df.shape}")
# head
big_df.head()

Shape: (737, 9)


,description,url,Street,R.,m²,Floor,Series,"Price, m2",Price
0,2 storey flat in a renovated house. Comes full...,https://ss.com/msg/en/real-estate/flats/riga/c...,Birznieka-Upisha 1..,4,108,5/6,Recon.,11.11 €,"1,200 €/mon."
1,"Fasādes māja, bezmaksas stāvvieta pagalmā, iee...",https://ss.com/msg/en/real-estate/flats/riga/c...,Pernavas 11,2,65,6/7,Pre-war house,5.38 €,350 €/mon.
2,"Fasādes māja, slēgts pagalms, ieeja no ielas u...",https://ss.com/msg/en/real-estate/flats/riga/c...,Baznicas 45,2,50,1/6,Pre-war house,5.20 €,260 €/mon.
3,Īrei tiek piedāvāts 3 istabu dzīvoklis Rīgas c...,https://ss.com/msg/en/real-estate/flats/riga/c...,Marijas 16,3,90,2/6,Pre-war house,7.78 €,700 €/mon.
4,Īrei tiek piedāvāts moderns dzīvoklis eklektis...,https://ss.com/msg/en/real-estate/flats/riga/c...,Stabu 30,2,53,3/4,Pre-war house,9.81 €,520 €/mon.


In [72]:
# now let's save to excel with datetime in the name
now = datetime.now()
dt_string = now.strftime("%Y%m%d_%H%M%S")# a little trick to add datetime to file name
# filename = f"aganskalns_{dt_string}.xlsx"
filename = f"centrs_{dt_string}.xlsx"
print(f"Filename: {filename}")
big_df.to_excel(filename)

Filename: centrs_20251204_153721.xlsx


In [73]:
# let us download the file
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# I can save files in my Google Drive as well do not have to download
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# so above approach works for static pages
# if we have dynamic pages we need to use selenium - which launches a browser controlled by python
# the methods are quite similar but we need to use selenium to get the response


In [ ]:
tokens = centrs.split("/")
tokens

['https:',
 '',
 'www.ss.com',
 'en',
 'real-estate',
 'flats',
 'riga',
 'centre',
 'sell',
 '']

In [ ]:
prefix = tokens[-3]+"_"+tokens[-2]
prefix

'centre_sell'

In [ ]:
# little aside on how to create file name with date and time in it
from datetime import datetime

In [ ]:
now = datetime.now()
now.year, now.month, now.day, now.hour, now.minute

(2022, 11, 30, 17, 41)

In [ ]:
fname = f"{prefix}_{now.month}_{now.day}_{ now.hour}_{ now.minute}.xlsx"
fname

'centre_sell_11_30_17_41.xlsx'

In [ ]:
datetime.timestamp()

<method 'timestamp' of 'datetime.datetime' objects>

In [ ]:
def getDFfromURL(url, save_to_excel=False):
    """
    url is a string representing a url of a page on ss.com
    save_to_excel is a boolean indicating if we want to save the data to excel file
    """
    # print("getting data from", url)
    req = requests.get(url)
    if req.status_code != 200: # check if we got a good response
        print("Request Fail with", req.status_code)
        return None # maybe return empty dataframe here
    soup = BeautifulSoup(req.text, 'lxml')
    column_names = getColList(soup) # so we utilize the functions we created earlier
    rowlist = getRowList(soup)
    rows = getRows(rowlist,colist=column_names)
    df = pd.DataFrame(rows, columns=column_names)
    if save_to_excel:
        tokens = url.split("/") # split url by / to get the prefix for the file name
        prefix = tokens[-3]+"_"+tokens[-2] # we get the prefix from the url
        now = datetime.now() # get current date and time
        fname = f"{prefix}_{now.month}_{now.day}_{now.hour}_{now.minute}.xlsx"
        df.to_excel(fname)
    return df

In [ ]:
centrs

'https://www.ss.com/en/real-estate/flats/riga/centre/sell/'

In [ ]:
agens = "https://www.ss.com/lv/real-estate/flats/riga/agenskalns/hand_over/"
agens

'https://www.ss.com/lv/real-estate/flats/riga/agenskalns/hand_over/'

In [ ]:
agens_df = getDFfromURL(agens, save_to_excel=True)
agens_df.head(10)  # see first 10 ads

,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
0,Ilgtermiņa īrei pieejams 3-istabu dzīvoklis Āg...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Valentīna 16,3,68,2/4,Renov.,8.46 €,575 €/mēn.
1,Īpašnieks ilgtermiņā izīrē mēbelētu dzīvokli Ā...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Gardenes 7,1,31,2/5,Hrušč.,10.32 €,320 €/mēn.
2,Tiek iznomāts pilnīgi jauns 2-istabu dzīvoklis...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Raņķa d. 34,2,40,8/24,Jaun.,20 €,800 €/mēn.
3,Izīrē dzīvokli ar daļējām ērtībām par 110.0€ m...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Augļu 4,1,16,1/2,Priv. m.,6.88 €,110 €/mēn.
4,"Izīre 3 istabu, +ēdamistaba, dzīvokli tuvajā Ā...",https://ss.com/msg/lv/real-estate/flats/riga/a...,Nometņu 9,3,66,2/4,P. kara,6.06 €,400 €/mēn.
5,Izīrēju dzīvokli Nometņu un Meža ielas krustoj...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Nometņu 1,2,60,2/3,P. kara,9.63 €,578 €/mēn.
6,Izīrēju ilgtermiņā 2-istabu dzīvokli Āgenskaln...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Kristapa 12,2,41,4/5,Hrušč.,7.32 €,300 €/mēn.
7,Izīrējam izremontētu un apmēbelētu 1-istabu dz...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Kokles 12B,1,33,2/4,Renov.,10.61 €,350 €/mēn.
8,Izīrējam izremontētu un apmēbelētu 2-istabu dz...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Kokles 12B,2,51,1/4,Renov.,7.45 €,380 €/mēn.
9,Izīrējam izremontētu un apmēbelētu 1-istabu dz...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Kokles 12B,1,21,3/4,Renov.,18.10 €,380 €/mēn.


In [ ]:
agens_sell = getDFfromURL("https://www.ss.com/lv/real-estate/flats/riga/agenskalns/sell/")
agens_sell.head(10)

,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
0,Tiek pārdots 2- līmeņu dzīvoklis Āgenskalnā. D...,https://ss.com/msg/lv/real-estate/flats/riga/a...,M. Nometņu 1,5,160,2/8,Specpr.,"1,406 €","225,000 €"
1,"Квартира с уникальным и красивым дизайном, кот...",https://ss.com/msg/lv/real-estate/flats/riga/a...,M. Nometņu 5,1,30,1/4,P. kara,"2,663 €","79,900 €"
2,Mūkusalas 56 ir pilnībā renovēta māja ar sakār...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Mūkusalas 56,2,62,1/3,Renov.,"1,315 €","81,500 €"
3,"Pārdod gaišu un saulainu, daļēji izremontētu 2...",https://ss.com/msg/lv/real-estate/flats/riga/a...,Tapešu 24,2,48,5/5,LT proj.,"1,096 €","52,600 €"
4,Pārdod Īres tiesības labi izremontētam 1 ist. ...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Talsu 30A,1,35,4/5,Priv. m.,180 €,"6,300 €"
5,Divistabu dzīvoklis ar balkonu jaunajā projekt...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Liepājas 2,2,64,3/6,Jaun.,"2,594 €","166,000 €"
6,Pārdod dzīvokli Āgenskalna centrā. Dzīvoklis a...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Zeļļu 13,2,47,1/4,P. kara,"1,056 €","49,630 €"
7,Mūkusalas 56 mājai ir jaunizbūvēts siltummezgl...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Mūkusalas 56,2,40,2/3,Renov.,"2,388 €","95,500 €"
8,Dzīvoklis ar baltu apdari (izņemot vannas ista...,https://ss.com/msg/lv/real-estate/flats/riga/a...,Āgenskalna 20,1,33,4/5,LT proj.,812 €,"26,800 €"
9,"Īpašniece pārdod saulainu, izremontētu 1-istab...",https://ss.com/msg/lv/real-estate/flats/riga/a...,Valguma 6-9,1,26,3/3,P. kara,"1,154 €","30,000 €"


In [ ]:
df = getDFfromURL(centrs, save_to_excel=True)

In [ ]:
df.shape

(30, 9)

In [ ]:
df.head()

,description,url,Street,R.,m2,Floor,Series,"Price, m2",Price
0,Divistabu dzīvoklis ar bezmaksas stāvvietu pie...,https://ss.com/msg/en/real-estate/flats/riga/c...,Miera 97,2,39,4/5,Chrusch.,8.46 €,330 €/mon.
1,Izīrē pilnībā mēbelētu dzīvokli jaunāja projek...,https://ss.com/msg/en/real-estate/flats/riga/c...,Terbatas 26a,2,31,2/5,New,14.52 €,450 €/mon.
2,Izīrē pilnībā mēbelētu dzīvokli pilsētas centr...,https://ss.com/msg/en/real-estate/flats/riga/c...,Terbatas 26a,2,43,1/6,New,10.47 €,450 €/mon.
3,Izīrē 2-istabu dzīvokli pēc kapitālā remonta a...,https://ss.com/msg/en/real-estate/flats/riga/c...,Chaka 148,2,60,3/5,Perewar,7.50 €,450 €/mon.
4,"Īpašniece piedāvā īrei ērtu, studijas tipa dzī...",https://ss.com/msg/en/real-estate/flats/riga/c...,Artilerijas 19,1,35,3/4,Recon.,10.57 €,370 €/mon.


## Now we want to find all pages with the same ads from the area
### some urls might have many ads some might have a single page

In [ ]:

# https://developer.mozilla.org/en-US/docs/Web/HTML/Element/a
anchors = soup.find_all("a")
len(anchors)
# anchors are the basic building blocks of the web, so we have a lot of them

99

In [ ]:
anchors[:5]

[<a href="/en/" title="Advertisements"><img alt="Advertisements" border="0" class="page_header_logo" src="https://i.ss.com/img/p.gif"/></a>,
 <a class="a_menu" href="/en/real-estate/flats/new/" title="Post Advertisement">Post Advertisement</a>,
 <a class="a_menu" href="/en/login/" title="My Adverts">My Adverts</a>,
 <a class="a_menu" href="/en/real-estate/flats/riga/centre/search/" title="Search advertisement">Search</a>,
 <a class="a_menu" href="/en/favorites/" title="Favorites">Favorites</a>]

In [ ]:
# we want the previous anchor it has a couple of attributes that we can use to find it
# one way is to use name attribute and check if it is equal to nav_id
navs = [anchor for anchor in anchors if anchor.get("name") == "nav_id"] # get will get us None if no "name" exists
len(navs)

10

In [ ]:
navs[0] # so the first one is the one we want

<a class="navi" href="/en/real-estate/flats/riga/centre/sell/page26.html" name="nav_id" rel="prev"><img border="0" height="5" src="https://i.ss.com/img/s_left.png" style="padding-bottom:2px;" width="9"/> Previous</a>

In [ ]:
navs[0].attrs

{'name': 'nav_id',
 'rel': ['prev'],
 'class': ['navi'],
 'href': '/en/real-estate/flats/riga/centre/sell/page26.html'}

In [ ]:
navs[0]['href'] # we can extract the link from the anchor

'/en/real-estate/flats/riga/centre/sell/page26.html'

In [ ]:
# could use regular expression here but we can use split
afterpage = navs[0]['href'].split("/page")[-1] # -1 might be safer than 1 in case there is region with name page
afterpage

'26.html'

In [ ]:
beforedot = afterpage.split(".html")[0] # first part of the split
beforedot

'26'

In [ ]:
# we have extracted the number of the page finally we need to convert it to integer
lastpage = int(beforedot)  # need to cast this to integer
lastpage

26

In [ ]:
# now we can use that number to create a list of urls for all pages

In [ ]:
def getAllLocalUrls(url):
    """Get a list of all urls including paginated pages
    url is a string representing a url of a page on ss.com where we start
    """
    results = [url] # default is just the url if no extra pages found like Boldaraja :)
    req = requests.get(url)
    if req.status_code != 200:
        print(f"Bad response! {req.status_code}")
        return []
    soup = BeautifulSoup(req.text, 'lxml')
    # we just need a one element
    # instead of name attribute we use rel attribute
    prevanchor = soup.find('a', {"rel":"prev"}) # this is specific to previous page anchor on ss.com
    # you would need to adjust this to your needs
    if prevanchor == None: # means there is only one page of ads
        return results
    # now we extact the number of the last page
    href = prevanchor.attrs.get('href')
    lastPageNum = int(href.split('/page')[-1].split('.html')[0])
    print("Last page is",lastPageNum)
    # finally we use list comprehension to create a list of urls
    # also note the use of f-string to create the url
    nurls = [f"{url}page{n}.html" for n in range(2,lastPageNum+1)]
    results += nurls
    return results

In [ ]:
centrs

'https://www.ss.com/en/real-estate/flats/riga/centre/sell/'

In [ ]:
local_urls = getAllLocalUrls(centrs)
len(local_urls),local_urls[:3],local_urls[-3:]

Last page is 26


(26,
 ['https://www.ss.com/en/real-estate/flats/riga/centre/sell/',
  'https://www.ss.com/en/real-estate/flats/riga/centre/sell/page2.html',
  'https://www.ss.com/en/real-estate/flats/riga/centre/sell/page3.html'],
 ['https://www.ss.com/en/real-estate/flats/riga/centre/sell/page24.html',
  'https://www.ss.com/en/real-estate/flats/riga/centre/sell/page25.html',
  'https://www.ss.com/en/real-estate/flats/riga/centre/sell/page26.html'])

In [ ]:
dlast = getDFfromURL(local_urls[-1])
dlast.shape

(9, 9)

In [ ]:
dlast

,description,url,Street,R.,m2,Floor,Series,"Price, m2",Price
0,"Īpašnieks pārdod mājīgu, saulainu un romantisk...",https://ss.com/msg/en/real-estate/flats/riga/c...,Sporta 7,2,51,1/4,Pre-war house,"2,353 €","120,000 €"
1,"Tiek pārdots dzīvoklis, ar privātmājas sajūtu ...",https://ss.com/msg/en/real-estate/flats/riga/c...,Chaka 62b,3,53,1/2,Pre-war house,"1,226 €","65,000 €"
2,Māja pēc pilnas rekonstrukcijas. Daļu no mājas...,https://ss.com/msg/en/real-estate/flats/riga/c...,Pulkv. Briezha 13,3,69,6/7,Recon.,"2,884 €","199,000 €"
3,Pārdodu saulainu dzīvokli jaunā projekta ēkā K...,https://ss.com/msg/en/real-estate/flats/riga/c...,Klijanu 2A,1,40,6/9,Recon.,950 €,"38,000 €"
4,"Īpašnieks pārdod dzīvokli, blakus Vef tilts, ļ...",https://ss.com/msg/en/real-estate/flats/riga/c...,Brivibas 181,2,42,2/5,Chrusch.,"1,857 €","78,000 €"
5,"Plašs, gaišs dzīvoklis ar elegantu plānojumu. ...",https://ss.com/msg/en/real-estate/flats/riga/c...,Skolas 30,4,115,3/5,Pre-war house,"2,000 €","230,000 €"
6,Labākais šāda veida piedāvājums centrā. \r\nKl...,https://ss.com/msg/en/real-estate/flats/riga/c...,Blaumana 21,6,166,3/5,Pre-war house,"1,627 €","270,000 €"
7,Pārdod 3 istabu dzīvokli projektā “Olive”. Zaļ...,https://ss.com/msg/en/real-estate/flats/riga/c...,Asara 9,3,109,4/6,New,"2,200 €","239,800 €"
8,"Mēbelēts dzīvoklis ar divām istabām, kur viena...",https://ss.com/msg/en/real-estate/flats/riga/c...,Barona 140a,2,27,2/2,Pre-war house,"1,000 €","27,000 €"


In [ ]:
def get_all_ads_df(start_url, save_excel_path=None):
    """Get a dataframe of all ads from a given url
    start_url is a string representing a url of a page on ss.com where we start
    save_excel_path is a string representing a path to save the excel file
    """
    df_list=[] # so we will save our dataframes in a list
    local_urls = getAllLocalUrls(start_url)
    for url in local_urls:
        print(f"Gathering data from {url}")
        df_list.append(getDFfromURL(url))
        time.sleep(0.3) # we need this to play nice! we want a little bit of delay (300ms here) to not overload the server
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html
    big_df = pd.concat(df_list) # then make a big dataframe from all the small dataframes - tables
    if save_excel_path:
        big_df.to_excel(save_excel_path)
    return big_df


In [ ]:
centrs

'https://www.ss.com/en/real-estate/flats/riga/centre/sell/'

In [ ]:
df = get_all_ads_df(centrs, "centrs_30_11_2022.xlsx")

Last page is 26
Gathering data from https://www.ss.com/en/real-estate/flats/riga/centre/sell/
Gathering data from https://www.ss.com/en/real-estate/flats/riga/centre/sell/page2.html
Gathering data from https://www.ss.com/en/real-estate/flats/riga/centre/sell/page3.html
Gathering data from https://www.ss.com/en/real-estate/flats/riga/centre/sell/page4.html
Gathering data from https://www.ss.com/en/real-estate/flats/riga/centre/sell/page5.html
Gathering data from https://www.ss.com/en/real-estate/flats/riga/centre/sell/page6.html
Gathering data from https://www.ss.com/en/real-estate/flats/riga/centre/sell/page7.html
Gathering data from https://www.ss.com/en/real-estate/flats/riga/centre/sell/page8.html
Gathering data from https://www.ss.com/en/real-estate/flats/riga/centre/sell/page9.html
Gathering data from https://www.ss.com/en/real-estate/flats/riga/centre/sell/page10.html
Gathering data from https://www.ss.com/en/real-estate/flats/riga/centre/sell/page11.html
Gathering data from http

In [ ]:
df.shape

(759, 9)

In [ ]:
df.head()

,description,url,Street,R.,m2,Floor,Series,"Price, m2",Price
0,Īpašnieks pārdod mūra-koka mājā vienistabas dz...,https://ss.com/msg/en/real-estate/flats/riga/c...,Petersalas 17,2,30,2/2,Spec. pr.,622 €,"18,650 €"
1,Lieliska iespēja iegādāties šo 2 istabu dzīvok...,https://ss.com/msg/en/real-estate/flats/riga/c...,Artilerijas 19,2,41,4/5,Pre-war house,"1,927 €","79,000 €"
2,Pārdod vienistabas dzīvokli ar lielisku plānoj...,https://ss.com/msg/en/real-estate/flats/riga/c...,Aluksnes 1,1,31,1/5,Recon.,"2,255 €","69,900 €"
3,Реновированный дом. Все удобства. Ремонт. Бесп...,https://ss.com/msg/en/real-estate/flats/riga/c...,Merkela 7,1,12,1/6,Pre-war house,898 €,"10,777 €"
4,Двухкомнатная квартира с парковочным местом во...,https://ss.com/msg/en/real-estate/flats/riga/c...,Valdemara 81,2,48,2/5,Pre-war house,"1,927 €","92,500 €"


In [ ]:
# so the steps of the process are:
# 1. get the list of urls for all pages
# 2. get the data from each page
# 3. combine the data into one dataframe
# 4. save the dataframe to excel for further analysis

In [ ]:
# what is the limitation of BeautifulSoup?
# it is not a browser, it does not execute javascript!
# modern web pages are not just html, they are html + css + javascript quite often
# often the page is not fully loaded until javascript is executed
# that is where Selenium comes in
# Selenium is a browser automation tool
# this means you can use it to automate the browser
# that is more powerful than BeautifulSoup but slower

In [ ]:
df.tail()

,description,url,Street,R.,m2,Floor,Series,"Price, m2",Price
3,Сдаётся однокомнатная квартира со всеми удобст...,https://ss.com/msg/en/real-estate/flats/riga/c...,Artilerijas 44,1,34,2/5,Pre-war house,6.18 €,210 €/mon.
4,"Izīrē jaunajā projektā komfortablu, kompaktu u...",https://ss.com/msg/en/real-estate/flats/riga/c...,Matisa 46,1,18,5/5,New,1.39 €,25 €/day
5,"Izīrē īstermiņā, Vecrīga - ideāla atrašanās vi...",https://ss.com/msg/en/real-estate/flats/riga/c...,Tirgonu 17,2,67,4/5,Pre-war house,0.746 €,50 €/day
6,"Rīgas centrs - ideāla atrašanās vieta, moderns...",https://ss.com/msg/en/real-estate/flats/riga/c...,Barona 24/26,2,46,6/6,Recon.,0.978 €,45 €/day
7,"Istaba īsire. ilgtermiņā. virtuve, tualete un ...",https://ss.com/msg/en/real-estate/flats/riga/c...,Chaka 32,1,15,6/6,Pre-war house,1.33 €,20 €/day


In [ ]:
df.shape

(608, 9)

In [ ]:
#we can do more post processing - feature engineering using existing columns
df[['CurFloor','MaxFloor']] = df.Floor.str.split("/",expand=True)
df.head()

,description,url,Street,R.,m2,Floor,Series,"Price, m2",Price,CurFloor,MaxFloor
0,Izīrē 2-istabu dzīvokli ilgtermiņa īrei Rīgas ...,https://ss.com/msg/en/real-estate/flats/riga/c...,Alunana 6,2,50,2/4,Pre-war house,8.60 €,430 €/mon.,2,4
1,Izīrējam ļoti mājīgu un plašu 90 m2 3-istabu d...,https://ss.com/msg/en/real-estate/flats/riga/c...,Ozolu 2,3,90,1/1,Priv.house,4.22 €,380 €/mon.,1,1
2,"Izīrējam dzīvokli, klusais centrs Vēstniecību ...",https://ss.com/msg/en/real-estate/flats/riga/c...,Dzirnavu 6,3,60,2/5,Recon.,11.67 €,700 €/mon.,2,5
3,"Fasādes māja, bezmaksas stāvvieta pagalmā. \r\...",https://ss.com/msg/en/real-estate/flats/riga/c...,Pernavas 11,2,65,6/7,Pre-war house,4.92 €,320 €/mon.,6,7
4,Izīrēju skaistu dzīvokli jaunā projekta ēkā. V...,https://ss.com/msg/en/real-estate/flats/riga/c...,Klusa 18,1,25,9/9,Recon.,1 €,25 €/day,9,9


In [ ]:
df.sort_values(by="Floor",ascending=False).head()

,description,url,Street,R.,m2,Floor,Series,"Price, m2",Price,CurFloor,MaxFloor
4,Izīrēju skaistu dzīvokli jaunā projekta ēkā. V...,https://ss.com/msg/en/real-estate/flats/riga/c...,Klusa 18,1,25,9/9,Recon.,1 €,25 €/day,9,9
21,For rent 2-bedroom apartment in a new building...,https://ss.com/msg/en/real-estate/flats/riga/c...,Grostonas 21,3,87,7/9,New,11.49 €,"1,000 €/mon.",7,9
29,Пентхаус с террасой и видом на город. \r\nKрас...,https://ss.com/msg/en/real-estate/flats/riga/c...,Pulkv. Briezha 21,5,265,7/7,New,11.32 €,"3,000 €/mon.",7,7
1,"Мансарда. Центр. Своё отопление. \r\nКрасиво, ...",https://ss.com/msg/en/real-estate/flats/riga/c...,Chaka 33,3,100,7/7,Pre-war house,9 €,900 €/mon.,7,7
20,"Izīrējam dzīvokli Rīgas pilsētas centrā - ""Upī...",https://ss.com/msg/en/real-estate/flats/riga/c...,Marijas 16,2,47,7/7,Recon.,11.70 €,550 €/mon.,7,7


In [ ]:
df.to_excel("c:/temp/my_apartments.xlsx") # i could this with absolute path

In [ ]:
from datetime import datetime as dt
now = dt.now()
now.month, now.day, now.hour, now.second

In [ ]:
url = "https://www.ss.com/lv/real-estate/flats/riga/sarkandaugava/hand_over/"
region = url.split("riga/")[-1].split("/")[0]
now = dt.now()
save_path = f"../data/{region}_{now.day}_{now.month}__{now.hour}_{now.second}.xlsx"
save_path

In [ ]:
url = "https://www.ss.com/lv/real-estate/flats/riga/sarkandaugava/hand_over/"
region = url.split("riga/")[-1].split("/")[0]
now = dt.now()
save_path = f"../data/{region}_{now.day}_{now.month}__{now.hour}_{now.second}.xlsx"

df = get_all_ads_df(url, save_path)

In [ ]:
url = "https://www.ss.com/lv/transport/cars/vaz/riga_f/"
df = get_all_ads_df(url, "../data/vaz_25_11.xlsx")

In [ ]:
sarkandaugava_url = "https://www.ss.com/lv/real-estate/flats/riga/sarkandaugava/hand_over/"
sarkandaugava_url

'https://www.ss.com/lv/real-estate/flats/riga/sarkandaugava/hand_over/'

In [ ]:
sarkan_df = get_all_ads_df(sarkandaugava_url, "sarkandaugava_01_12_21.xlsx")
sarkan_df

Last page is 2
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/sarkandaugava/hand_over/
Gathering data from https://www.ss.com/lv/real-estate/flats/riga/sarkandaugava/hand_over/page2.html


,description,url,Iela,Ist.,m2,Stāvs,Sērija,"Cena, m2",Cena
0,Saimniece izīrē divistaba dzīvokli ar malkas a...,https://ss.com/msg/lv/real-estate/flats/riga/s...,Ziemeļu 17,2,50,4/4,P. kara,4.80 €,240 €/mēn.
1,Сдается однокомнатная квартира со всеми удобст...,https://ss.com/msg/lv/real-estate/flats/riga/s...,Tilta 5/1,1,34,5/5,103.,6.47 €,220 €/mēn.
2,"Квартира в сталинском доме, высокий первый эта...",https://ss.com/msg/lv/real-estate/flats/riga/s...,Sarkandaugavas 3,2,44,1/5,Staļina,6.82 €,300 €/mēn.
3,"Izīrē dzīvokli ar visām ērtībām, tehniku, dīvā...",https://ss.com/msg/lv/real-estate/flats/riga/s...,Sarkandaugavas 26,2,43,5/5,Hrušč.,5.81 €,250 €/mēn.
4,Izīrē ērtu dzīvokli ar remontu un mēbelēm. Krā...,https://ss.com/msg/lv/real-estate/flats/riga/s...,Priežu 14a,1,27,2/3,P. kara,5.93 €,160 €/mēn.
5,"Владелец сдаёт мебелированную, со всеми удобст...",https://ss.com/msg/lv/real-estate/flats/riga/s...,Viestura pr. 35,2,47,5/9,Specpr.,5.96 €,280 €/mēn.
6,Tikko pēc kosmētiska remonta. Istabas caurstai...,https://ss.com/msg/lv/real-estate/flats/riga/s...,Patversmes 26,2,41,2/5,Hrušč.,7.80 €,320 €/mēn.
7,"Izīrē 2istabu dzīvokli Sarkandaugavā, Limbažu ...",https://ss.com/msg/lv/real-estate/flats/riga/s...,Limbažu 9,2,40,1/5,Hrušč.,5 €,200 €/mēn.
8,Izīrēju divistaba dzīvokli ar daļējām ērtībām ...,https://ss.com/msg/lv/real-estate/flats/riga/s...,Ziemeļu 17,2,36,3/4,P. kara,5 €,180 €/mēn.
9,"Uz ilgtermiņa īri pieejams saulains, remontēts...",https://ss.com/msg/lv/real-estate/flats/riga/s...,Duntes 54,2,50,2/3,P. kara,6.60 €,330 €/mēn.


# TODO
# Try with different starting address not only centrs
## Maybe combine regions
## See how it would work with maybe cars
## Data engineering make new columns based on existing ones, clean some columns
### Changing floors 2/6 to columns 2 and 6, clean up Euro signs
## Sorting, Describing, Grouping by regions etc

In [ ]:
df = pd.read_excel("centrs_25_11.xlsx")
df.head()

In [ ]:
df["price"] = df.Cena.str.split(' ').str[0]
df.head()

In [ ]:
df.price = df.price.str.replace(",","").astype('int32')
df.head()

In [ ]:
df.price.hist()

In [ ]:
df[['Floor','MaxFloor']] = df.Stāvs.str.split("/",expand=True)
df.info()

In [ ]:
#typecasting example
df.Floor = df.Floor.astype('int32')
df.MaxFloor = df.MaxFloor.astype('int32')

In [ ]:
df.describe().T # T is for transposing rows and columns

In [ ]:
df.describe().T.to_csv("center_stats.csv")

In [ ]:
import plotly.express as px

In [ ]:
fig = px.histogram(df, x="price")
fig.show()

In [ ]:
import seaborn as sns # seaborn sits on top of matplotlib with better defaults
import matplotlib.pyplot as plt


In [ ]:
# Basic correlogram
sns.pairplot(df)



In [ ]:
# Basic correlogram
sns.pairplot(df)



In [ ]:
volvo_url = "https://www.ss.com/lv/transport/cars/volvo/sell/"
gr_auto = df.groupby(["Gads","Modelis"])["Cena"].agg(["min", "max", "mean"])  #.head(n = 10)


In [ ]:
print(dt.today())

In [ ]:
suzuki_url = "https://www.ss.com/lv/transport/cars/suzuki/"
# sdf = get_all_ads_df(suzuki_url, f"suzuki_{dt.today()}.xlsx")
sdf = get_all_ads_df(suzuki_url, f"suzuki.xlsx")
sdf.head()

In [ ]:
sdf.info()

In [ ]:
sdf["Tilp."] = sdf["Tilp."].str.replace("\w","", regex=True)
sdf.head()

In [ ]:
sdf["Nobrauk."] = sdf["Nobrauk."].str.replace("\D","", regex=True) # so no digits removed
sdf.head()

In [ ]:
sdf["Cena"] = sdf["Cena"].str.replace("\D","", regex=True) # so no digits removed
sdf.head()

In [ ]:
sdf = sdf.dropna(how="any", axis=0)
sdf

In [ ]:
sdf = sdf[sdf["Nobrauk."].str.len() > 0] # got rid of those empty rows finally
sdf

In [ ]:
sdf["Cena"] = sdf["Cena"].astype("float32")

In [ ]:
gr_auto = sdf.groupby(["Gads","Modelis"])["Cena"].agg(["count","min", "max", "mean"])
gr_auto

## Selenium drivers for when Beatiful Soup down not wok

SOmetimes pages are not static, (especially if the page is not fully built).. Then you would use Selenium.

### Selenium Drivers for Python

In case of Selenium you need to install Selenium itself and also drivers for a particular Web Browser.

So if you are using Chrome you owuld install drivers for Chrome.

So Selenium gets past other pages which are stricter for requiring Javascript and CSS and so on.

https://www.selenium.dev/selenium/docs/api/py/